In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max as spark_max, countDistinct, sum as spark_sum, datediff, lit, when
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Initialize Spark session
spark = SparkSession.builder \
    .appName("CustomerJourneyAnalysis") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.1") \
    .config("fs.azure.account.key.dataintensiveproject.blob.core.windows.net", os.getenv("AZURE_STORAGE_KEY")) \
    .getOrCreate()


Python-dotenv could not parse statement starting at line 2
Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 4
25/04/21 23:27:43 WARN Utils: Your hostname, Rishabhs-Laptop.local resolves to a loopback address: 127.0.0.1; using 192.168.0.179 instead (on interface en0)
25/04/21 23:27:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/rishabhsaudagar/.ivy2/cache
The jars for the packages stored in: /Users/rishabhsaudagar/.ivy2/jars
org.apache.hadoop#hadoop-azure added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-973b49d1-e6fd-4426-bc52-99f1fe86dadf;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hadoop#hadoop-azure;3.3.1 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.11 in central
	found com.microsoft.azure#azure-storage;7.0.1 in central
	found com.fasterxml.jackson.core#jackson-core;2.10.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found com.microsoft.azure#azure-keyvault-core;1.0.0 in central
	found com.google.guava#guava;27.0-jre in central
	found com.google.guava#failureaccess;1.0 in central
	found com.google.guava#listenablefuture;9999.0-empty-to-avoid-conflict-with-guava in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.5.2 in central
	found com.google.errorprone#error_prone_annotations;2.2.0 in central
	found com.google.j2objc#j2objc-annotations;1.1 in central
	found org.codehaus.mojo#animal-sniffer-anno

In [4]:
# Path to your Parquet data
parquet_path = "wasbs://retail-data@dataintensiveproject.blob.core.windows.net/clean/retail_cleaned"

# Read the cleaned data
df = spark.read.parquet(parquet_path)
df.printSchema()
df.show(5)


25/04/21 23:27:45 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-azure-file-system.properties,hadoop-metrics2.properties


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- InvoiceDate: timestamp_ntz (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- InvoiceYear: integer (nullable = true)
 |-- InvoiceMonth: integer (nullable = true)
 |-- InvoiceWeekday: integer (nullable = true)
 |-- InvoiceHour: integer (nullable = true)



+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+------------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|InvoiceYear|InvoiceMonth|InvoiceWeekday|InvoiceHour|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+------------+--------------+-----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|       2010|          12|             4|          8|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|       2010|          12|             4|          8|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|       2010|          12|             4|          8|
|   536365|   84029G|KNITTED UNION FLA..

In [5]:
# Create a new column: Quantity × UnitPrice
df = df.withColumn("TotalRevenue", col("Quantity") * col("UnitPrice"))
df.select("InvoiceNo", "CustomerID", "TotalRevenue").show(5)


+---------+----------+------------------+
|InvoiceNo|CustomerID|      TotalRevenue|
+---------+----------+------------------+
|   536365|   17850.0|15.299999999999999|
|   536365|   17850.0|             20.34|
|   536365|   17850.0|              22.0|
|   536365|   17850.0|             20.34|
|   536365|   17850.0|             20.34|
+---------+----------+------------------+
only showing top 5 rows



In [6]:
# Use latest invoice date as the snapshot date
snapshot_date = df.agg(spark_max("InvoiceDate")).first()[0]
print("Snapshot date:", snapshot_date)


Snapshot date: 2011-12-09 12:50:00


In [7]:
# Group by CustomerID to get Recency, Frequency, and Monetary
rfm_df = df.groupBy("CustomerID").agg(
    spark_max("InvoiceDate").alias("LastPurchaseDate"),
    countDistinct("InvoiceNo").alias("Frequency"),
    spark_sum("TotalRevenue").alias("Monetary")
).withColumn("Recency", datediff(lit(snapshot_date), col("LastPurchaseDate")))

rfm_df.select("CustomerID", "Recency", "Frequency", "Monetary").show(10)


+----------+-------+---------+------------------+
|CustomerID|Recency|Frequency|          Monetary|
+----------+-------+---------+------------------+
|   13094.0|     21|       12|1927.3200000000002|
|   17884.0|      3|        4|            717.45|
|   16561.0|      5|        2|            511.12|
|   13973.0|    287|        1|             264.7|
|   14285.0|     21|        4|1910.0100000000002|
|   17633.0|     31|        4|           1324.84|
|   13956.0|      5|        5|           1026.42|
|   13533.0|    182|        3|270.78999999999996|
|   13918.0|     49|        2|           1212.84|
|   17267.0|    127|        2|317.61999999999995|
+----------+-------+---------+------------------+
only showing top 10 rows



In [8]:
# Create R, F, M scores
rfm_df = rfm_df.withColumn("R_Score", 
    when(col("Recency") <= 30, 3)
    .when(col("Recency") <= 90, 2)
    .otherwise(1)) \
    .withColumn("F_Score", 
    when(col("Frequency") >= 20, 3)
    .when(col("Frequency") >= 5, 2)
    .otherwise(1)) \
    .withColumn("M_Score", 
    when(col("Monetary") >= 1000, 3)
    .when(col("Monetary") >= 300, 2)
    .otherwise(1))

rfm_df.select("CustomerID", "R_Score", "F_Score", "M_Score").show(10)


+----------+-------+-------+-------+
|CustomerID|R_Score|F_Score|M_Score|
+----------+-------+-------+-------+
|   13094.0|      3|      2|      3|
|   17884.0|      3|      1|      2|
|   16561.0|      3|      1|      2|
|   13973.0|      1|      1|      1|
|   14285.0|      3|      1|      3|
|   17633.0|      2|      1|      3|
|   13956.0|      3|      2|      3|
|   13533.0|      1|      1|      1|
|   13918.0|      2|      1|      3|
|   17267.0|      1|      1|      2|
+----------+-------+-------+-------+
only showing top 10 rows



In [16]:
rfm_df.coalesce(1).write.mode('overwrite').option("header", True).csv("PowerBI/rfm_results")

25/04/22 05:02:38 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1037217 ms exceeds timeout 120000 ms
25/04/22 05:02:38 WARN SparkContext: Killing executors is not supported by current scheduler.
25/04/22 05:02:41 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at s

In [9]:
# Define output path for RFM data
output_path = "wasbs://retail-data@dataintensiveproject.blob.core.windows.net/transformed/customer_rfm"

# Write as Parquet to Azure Blob
rfm_df.write.mode("overwrite").parquet(output_path)

print("✅ RFM data saved to Azure Blob at: /transformed/customer_rfm")


25/04/21 23:27:53 WARN AzureFileSystemThreadPoolExecutor: Disabling threads for Delete operation as thread count 0 is <= 1
25/04/21 23:27:55 WARN AzureFileSystemThreadPoolExecutor: Disabling threads for Rename operation as thread count 0 is <= 1
25/04/21 23:27:56 WARN AzureFileSystemThreadPoolExecutor: Disabling threads for Delete operation as thread count 0 is <= 1


✅ RFM data saved to Azure Blob at: /transformed/customer_rfm


In [10]:
#save RFM to Azure SQL

# Define JDBC properties
jdbc_url = os.getenv("AZURE_SQL_URL")
table_name = "CustomerRFM"
jdbc_properties = {
    "user": os.getenv("AZURE_SQL_USER"),
    "password": os.getenv("AZURE_SQL_PASSWORD"),
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# Save to Azure SQL
rfm_df.write \
    .mode("overwrite") \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .save()

print("✅ RFM data saved to Azure SQL table: CustomerRFM")


Py4JJavaError: An error occurred while calling o127.save.
: java.lang.ClassNotFoundException: com.microsoft.sqlserver.jdbc.SQLServerDriver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOp

In [ ]:
from pyspark.sql import SparkSession
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Path to JDBC JAR
jdbc_jar_path = os.path.expanduser("~/.ivy2/jars/com.microsoft.sqlserver_mssql-jdbc-9.4.1.jre8.jar")

# Initialize Spark with JDBC driver
spark = SparkSession.builder \
    .appName("RetailETL") \
    .config("spark.jars", jdbc_jar_path) \
    .config("fs.azure.account.key.datainstensiveproject.blob.core.windows.net", os.getenv("AZURE_STORAGE_KEY")) \
    .getOrCreate()

Python-dotenv could not parse statement starting at line 2
Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 4
25/04/21 19:52:17 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
for jar in spark.sparkContext._jars:
    if "mssql-jdbc" in jar:
        print("✅ JDBC driver loaded:", jar)

AttributeError: 'SparkContext' object has no attribute '_jars'

In [ ]:
print("\n".join(spark.sparkContext._gateway.jvm.java.lang.System.getProperty("java.class.path").split(":")))

/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/pyspark/conf
/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/pyspark/jars/dropwizard-metrics-hadoop-metrics2-reporter-0.1.2.jar
/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/pyspark/jars/netty-handler-proxy-4.1.87.Final.jar
/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/pyspark/jars/logging-interceptor-3.12.12.jar
/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/pyspark/jars/spark-network-common_2.12-3.4.1.jar
/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/pyspark/jars/threeten-extra-1.7.1.jar
/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/pyspark/jars/hive-shims-2.3.9.jar
/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/pyspark/jars/spark-core_2.12-3.4.1.jar
/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/pyspark/jars/httpcore-4.4.16.jar
/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/pyspark/jars/jakarta.xml.bind-ap

In [ ]:
jdbc_url = os.getenv("AZURE_SQL_URL")
table_name = "CustomerRFM"

rfm_df.write \
    .mode("overwrite") \
    .format("jdbc") \
    .option("url", os.getenv("AZURE_SQL_URL")) \
    .option("dbtable", "CustomerRFM") \
    .option("user", os.getenv("AZURE_SQL_USER")) \
    .option("password", os.getenv("AZURE_SQL_PASSWORD")) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()


print("✅ RFM data written to Azure SQL.")

Py4JJavaError: An error occurred while calling o182.save.
: java.lang.ClassNotFoundException: com.microsoft.sqlserver.jdbc.SQLServerDriver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOp

In [ ]:
try:
    rfm_df.write \
        .mode("overwrite") \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", table_name) \
        .option("user", os.getenv("AZURE_SQL_USER")) \
        .option("password", os.getenv("AZURE_SQL_PASSWORD")) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .save()

    print("✅ RFM data written to Azure SQL.")

except Exception as e:
    import traceback
    traceback.print_exc()


Traceback (most recent call last):
  File "/var/folders/x6/m7520qdd4m1dts_g2lwmd0vc0000gn/T/ipykernel_59026/2917277426.py", line 2, in <module>
    rfm_df.write \
  File "/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/pyspark/sql/readwriter.py", line 1396, in save
    self._jwrite.save()
  File "/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/pyspark/errors/exceptions/captured.py", line 169, in deco
    return f(*a, **kw)
  File "/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: An error occurred while calling o171.save.
: java.lang.ClassNotFoundException: com.microsoft.sqlserver.jdbc.SQLServerDriver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassL